## Imports

In [2]:
from torchvision import datasets
from torchvision import transforms
import torch
import torch.nn as nn
import numpy as np

c:\Users\Charl\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
rootdir = '/Data_cfar10'
# Downloading Dataset as Tensors
T = datasets.CIFAR10(rootdir,train=True,download=True, transform=transforms.ToTensor())
V = datasets.CIFAR10(rootdir,train=False,download=True, transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [12]:
tL = torch.utils.data.DataLoader(T, batch_size=64, shuffle=True, drop_last=True)
vL = torch.utils.data.DataLoader(V, batch_size=64, shuffle=True, drop_last=True)

In [29]:
model = nn.Sequential(
    nn.Linear(3072, 100),
    nn.ReLU(),
    nn.Linear(100, 10),
    nn.LogSoftmax(dim=1)
)

In [21]:
loss_fn = nn.NLLLoss()

In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(tL))

In [31]:
nepochs = 2
for e in range(nepochs):
    eLoss = 0
    for X,y in tL:
        batch_size = X.shape[0]
        y_hat = model(X.view(batch_size,-1))
        loss = loss_fn(y_hat,y)
        eLoss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()
    print(float(eLoss))

1803.0543212890625
1803.0537109375


In [32]:
c = 0
t = 0
with torch.no_grad():
    for Xv,yv in vL:
        batch_size = Xv.shape[0]
        y_hat = model(Xv.view(batch_size,-1))
        _,p = torch.max(y_hat,dim=1)
        t+=yv.shape[0]
        c+=int((p==yv).sum())
print(c/t)

0.10256410256410256
